# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import sys

### Importing Data as pandas dataframe and preprocessing it

In [104]:
df = pd.read_csv(r"C:\Users\Aman-Verma\Downloads\ASP Transition Matrix and HMM\RP Data\NIFTY 2010-20.csv",error_bad_lines=False)

In [105]:
df = df.drop('High',axis=1)
df = df.drop('Low',axis=1)
df = df.drop('Adj Close',axis=1)
df = df.drop('Volume',axis=1)

In [106]:
df.describe()

,Open,Close
count,2445.000000,2445.000000
mean,7758.714319,7752.939513
std,2212.016988,2208.878781
min,4623.149902,4544.200195
25%,5681.100098,5677.899902
50%,7788.500000,7784.649902
75%,9657.150391,9657.549805
max,12274.900391,12271.799805


In [107]:
df.head()

,Date,Open,Close
0,2010-01-04,5200.899902,5232.200195
1,2010-01-05,5277.149902,5277.899902
2,2010-01-06,5278.149902,5281.799805
3,2010-01-07,5281.799805,5263.100098
4,2010-01-08,5264.250000,5244.750000


### Setting up the parameters for the model 

In [108]:
df['Difference'] = df['Close'] - df['Open']

In [109]:
df['Change Mod'] = df['Difference'].diff(-1).abs() / df['Open']

In [110]:
df.head()

,Date,Open,Close,Difference,Change Mod
0,2010-01-04,5200.899902,5232.200195,31.300293,0.005874
1,2010-01-05,5277.149902,5277.899902,0.750000,0.000550
2,2010-01-06,5278.149902,5281.799805,3.649903,0.004234
3,2010-01-07,5281.799805,5263.100098,-18.699707,0.000152
4,2010-01-08,5264.250000,5244.750000,-19.500000,0.000969


In [111]:
df.describe()

,Open,Close,Difference,Change Mod
count,2445.000000,2445.000000,2445.000000,2420.000000
mean,7758.714319,7752.939513,-5.774805,0.009146
std,2212.016988,2208.878781,62.811474,0.007936
min,4623.149902,4544.200195,-246.950195,0.000000
25%,5681.100098,5677.899902,-40.799804,0.003388
50%,7788.500000,7784.649902,-5.400390,0.007141
75%,9657.150391,9657.549805,31.549804,0.012750
max,12274.900391,12271.799805,527.400390,0.061576


### Setting up the range  

In [112]:
criteria = [df['Difference'].between(-1000, -df['Change Mod']*df['Open']/2), df['Difference'].between( -df['Change Mod']*df['Open']/2, df['Change Mod']*df['Open']/2), df['Difference'].between(df['Change Mod']*df['Open']/2,1000)]
values = ['Down', 'Stagnant', 'Up']

df['Movement'] = np.select(criteria, values, 'Stagnant')

In [113]:
df.head()

,Date,Open,Close,Difference,Change Mod,Movement
0,2010-01-04,5200.899902,5232.200195,31.300293,0.005874,Up
1,2010-01-05,5277.149902,5277.899902,0.750000,0.000550,Stagnant
2,2010-01-06,5278.149902,5281.799805,3.649903,0.004234,Stagnant
3,2010-01-07,5281.799805,5263.100098,-18.699707,0.000152,Down
4,2010-01-08,5264.250000,5244.750000,-19.500000,0.000969,Down


### Showing number of States  

In [114]:
df['Movement'].value_counts()

Stagnant    919
Down        842
Up          708
Name: Movement, dtype: int64

In [115]:
z = list(df['Movement'])

### Transition Probability Matrix 

In [116]:
pd.crosstab(pd.Series(z[1:],name='Tomorrow'),
            pd.Series(z[:-1],name='Today'),normalize=0)

Today,Down,Stagnant,Up
Tomorrow,,,
Down,0.372922,0.458432,0.168646
Stagnant,0.406964,0.241567,0.351469
Up,0.217822,0.438472,0.343706


In [117]:
from itertools import islice

def window(seq, n=2):
    "Sliding window width n from seq.  From old itertools recipes."""
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [118]:
import pandas as pd

pairs = pd.DataFrame(window(z), columns=['Today', 'Tomorrow'])
counts = pairs.groupby('Today')['Tomorrow'].value_counts()
probs = (counts).unstack()

### Transition Count Matrix

In [119]:
probs

Tomorrow,Down,Stagnant,Up
Today,,,
Down,314,374,154
Stagnant,386,222,310
Up,142,323,243


# End